<h1 align="center"><b>CS116.M11 - LẬP TRÌNH PYTHON CHO MÁY HỌC</b></h1>
<h1 align="center"><b>BÀI TẬP 11 - NGÀY 16/11/2021</b></h1>

# **Thông tin sinh viên**

1. ***MSSV:*** 19520214
2. ***Họ tên:*** Cao Hưng Phú
3. ***Bài tập:*** Bài tập 11
4. ***Ngày thực hiện:*** 16/11/2021

# **Bài làm**

### **Các hàm và thư viện cần thiết**

In [27]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

### **Bước 1: Load dữ liệu Social Network Ads**

In [28]:
# Tải file Social_Network_Ads.csv từ GoogleDrive
!gdown --id "1vlfarV6qq7ot4KXb9NW8RoG2Rm1cJ0E2"

# Gán path file Social_Network_Ads.csv
path_data = "/content/Social_Network_Ads.csv"

# Đọc file Wine.csv
df = pd.read_csv(path_data)

# Hiển thị 5 dòng đầu của data
df.head(5)

Downloading...
From: https://drive.google.com/uc?id=1vlfarV6qq7ot4KXb9NW8RoG2Rm1cJ0E2
To: /content/Social_Network_Ads.csv
100% 4.90k/4.90k [00:00<00:00, 3.71MB/s]


,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


### **Bước 2: Chuẩn hóa dữ liệu với Standard Scaler**

In [29]:
X = np.array(df[['Age', 'EstimatedSalary']])
y = np.array(df['Purchased'])

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

### **Bước 3: Làm theo yêu cầu đề bài**

#### **3.0: Liệt kê các siêu tham số của SVM và miền giá trị của từng tham số**

In [31]:
from sklearn.svm import SVC
model = SVC()
model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [32]:
listKernel = ['linear', 'poly', 'rbf', 'sigmoid']
listC = [0.01, 0.1, 1, 10, 100, 1000]
listGamma = [1, 0.1, 0.01, 0.001, 0.0001]

#### **3.1: Chia tập dataset train, val, test theo tỉ lệ 6:2:2**

In [33]:
import numpy as np
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [35]:
print("Data train:", len(X_train))
print("Data validation:", len(X_val))
print("Data test:", len(X_test))

Data train: 240
Data validation: 80
Data test: 80


#### **3.2: Tìm hyper-parameters tốt nhất**

##### **3.2.1: Tìm bằng for**

In [36]:
from sklearn.metrics import accuracy_score
bestAccuracy = 0
bestK, bestC, bestG = None, None, None
for k in listKernel:
  for c in listC:
    for g in listGamma:
      modelTMP = SVC(kernel=k, C=c, gamma=g)
      modelTMP.fit(X_train, y_train)
      y_predict = modelTMP.predict(X_val)
      accuracyTMP = accuracy_score(y_val, y_predict)
      print("Params: kernel = {}, C = {}, gamma = {}".format(k, c, g))
      print("=> Accurracy:", accuracyTMP)
      print("=======================================================")
      if accuracyTMP > bestAccuracy:
        bestAccuracy = accuracyTMP
        bestK, bestC, bestG = k, c, g

Params: kernel = linear, C = 0.01, gamma = 1
=> Accurracy: 0.7625
Params: kernel = linear, C = 0.01, gamma = 0.1
=> Accurracy: 0.7625
Params: kernel = linear, C = 0.01, gamma = 0.01
=> Accurracy: 0.7625
Params: kernel = linear, C = 0.01, gamma = 0.001
=> Accurracy: 0.7625
Params: kernel = linear, C = 0.01, gamma = 0.0001
=> Accurracy: 0.7625
Params: kernel = linear, C = 0.1, gamma = 1
=> Accurracy: 0.7875
Params: kernel = linear, C = 0.1, gamma = 0.1
=> Accurracy: 0.7875
Params: kernel = linear, C = 0.1, gamma = 0.01
=> Accurracy: 0.7875
Params: kernel = linear, C = 0.1, gamma = 0.001
=> Accurracy: 0.7875
Params: kernel = linear, C = 0.1, gamma = 0.0001
=> Accurracy: 0.7875
Params: kernel = linear, C = 1, gamma = 1
=> Accurracy: 0.7875
Params: kernel = linear, C = 1, gamma = 0.1
=> Accurracy: 0.7875
Params: kernel = linear, C = 1, gamma = 0.01
=> Accurracy: 0.7875
Params: kernel = linear, C = 1, gamma = 0.001
=> Accurracy: 0.7875
Params: kernel = linear, C = 1, gamma = 0.0001
=> Accurr

In [37]:
print("Best kernel: {} - Best C: {} - Best gamma: {} -> Accuracy validation: {}".format(bestK, bestC, bestG, bestAccuracy))

Best kernel: rbf - Best C: 100 - Best gamma: 0.1 -> Accuracy validation: 0.9125


##### **3.2.1: Tìm bằng thư viện**

In [38]:
from sklearn.model_selection import GridSearchCV
 
param_grid = {'kernel': listKernel, 'C': listC, 'gamma': listGamma}
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.792 total time=   0.0s
[CV 2/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.771 total time=   0.0s
[CV 3/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.708 total time=   0.0s
[CV 4/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.729 total time=   0.0s
[CV 5/5] END ....C=0.01, gamma=1, kernel=linear;, score=0.771 total time=   0.0s
[CV 1/5] END ......C=0.01, gamma=1, kernel=poly;, score=0.729 total time=   0.0s
[CV 2/5] END ......C=0.01, gamma=1, kernel=poly;, score=0.771 total time=   0.0s
[CV 3/5] END ......C=0.01, gamma=1, kernel=poly;, score=0.750 total time=   0.0s
[CV 4/5] END ......C=0.01, gamma=1, kernel=poly;, score=0.771 total time=   0.0s
[CV 5/5] END ......C=0.01, gamma=1, kernel=poly;, score=0.792 total time=   0.0s
[CV 1/5] END .......C=0.01, gamma=1, kernel=rbf;, score=0.646 total time=   0.0s
[CV 2/5] END .......C=0.01, gamma=1, kernel=rb

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [39]:
print("Best params:", grid.best_params_)
print("Best estimator:", grid.best_estimator_)

Best params: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Best estimator: SVC(C=1, gamma=1)


#### **3.3: Accuracy trên tập test**

##### **3.3.1: Accuracy trên tập test dùng for**

In [40]:
print("Best kernel: {} - Best C: {} - Best gamma: {} -> Accuracy validation: {}".format(bestK, bestC, bestG, bestAccuracy))

Best kernel: rbf - Best C: 100 - Best gamma: 0.1 -> Accuracy validation: 0.9125


In [41]:
myModel = SVC(kernel=bestK, C=bestC, gamma=bestG)
myModel.fit(X_train, y_train)
y_predict = myModel.predict(X_test)
myAccuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", myAccuracy)

Accuracy: 0.925


In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.98      0.90      0.94        52
           1       0.84      0.96      0.90        28

    accuracy                           0.93        80
   macro avg       0.91      0.93      0.92        80
weighted avg       0.93      0.93      0.93        80



##### **3.3.1: Accuracy trên tập test dùng thư viện**

In [43]:
print("Best params:", grid.best_params_)
print("Best estimator:", grid.best_estimator_)

Best params: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
Best estimator: SVC(C=1, gamma=1)


In [44]:
grid_predictions = grid.predict(X_test)
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.98      0.92      0.95        52
           1       0.87      0.96      0.92        28

    accuracy                           0.94        80
   macro avg       0.93      0.94      0.93        80
weighted avg       0.94      0.94      0.94        80

